# ADS 509 Assignment 2.1: Tokenization, Normalization, Descriptive Statistics 

This notebook holds Assignment 2.1 for Module 2 in ADS 509, Applied Text Mining. Work through this notebook, writing code and answering questions where required. 

In the previous assignment you pulled lyrics data on two artists. In this assignment we explore this data set and a pull from the now-defunct Twitter API for the artists Cher and Robyn.  If, for some reason, you did not complete that previous assignment, data to use for this assignment can be found in the assignment materials section of Canvas. 

This assignment asks you to write a short function to calculate some descriptive statistics on a piece of text. Then you are asked to find some interesting and unique statistics on your corpora. 


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


In [2]:
import os
import re
import emoji
import pandas as pd
import numpy as np

from collections import Counter, defaultdict
import nltk
from nltk.corpus import stopwords
from string import punctuation

sw = stopwords.words("english")

In [3]:
import string

In [4]:
# change `data_location` to the location of the folder on your machine.
data_location = "/Users/mtc/ADS/ADS 509/M1 Results"

# These subfolders should still work if you correctly stored the 
# data from the Module 1 assignment
twitter_folder = "twitter/"
lyrics_folder = "lyrics/"

In [5]:
ds_store_file_location = os.path.join(data_location, lyrics_folder, '.DS_store')
if os.path.isfile(ds_store_file_location):
    os.remove(ds_store_file_location)

ds_store_file_location = os.path.join(data_location, twitter_folder, '.DS_store')
if os.path.isfile(ds_store_file_location):
    os.remove(ds_store_file_location)

In [6]:
def descriptive_stats(tokens, num_tokens = 5, verbose=True) :
    """
        Given a list of tokens, print number of tokens, number of unique tokens, 
        number of characters, lexical diversity (https://en.wikipedia.org/wiki/Lexical_diversity), 
        and num_tokens most common tokens. Return a list with the number of tokens, number
        of unique tokens, lexical diversity, and number of characters. 
    """

    # Fill in the correct values here. 
    num_tokens = len(tokens)
    num_unique_tokens = len(set(tokens))
    if num_tokens > 0:
        lexical_diversity = num_unique_tokens/num_tokens
    else: lexical_diversity = 0
    num_characters = sum(len(token) for token in tokens)
    
    if verbose :        
        print(f"There are {num_tokens} tokens in the data.")
        print(f"There are {num_unique_tokens} unique tokens in the data.")
        print(f"There are {num_characters} characters in the data.")
        print(f"The lexical diversity is {lexical_diversity:.3f} in the data.")
    
        # print the five most common tokens
        
    return([num_tokens, num_unique_tokens,
            lexical_diversity,
            num_characters])
    

In [6]:
text = """here is some example text with other example text here in this text""".split()
assert(descriptive_stats(text, verbose=True)[0] == 13)
assert(descriptive_stats(text, verbose=False)[1] == 9)
assert(abs(descriptive_stats(text, verbose=False)[2] - 0.69) < 0.02)
assert(descriptive_stats(text, verbose=False)[3] == 55)


There are 13 tokens in the data.
There are 9 unique tokens in the data.
There are 55 characters in the data.
The lexical diversity is 0.692 in the data.


Q: Why is it beneficial to use assertion statements in your code? 

A: Assertion statements are beneficial because if the requirement specified within the statement is not met, an error is returned, stopping the rest of the code from running. If only a bollean output, such as True or False, is raised when requirements are not met, the person running the code might miss that the requirement of the check was not met. By raising an error and stopping future code chunks from running, assertion statements help bring attention towards issues earlier. Finding issues earlier leads to less wasted time down the line.

## Data Input

Now read in each of the corpora. For the lyrics data, it may be convenient to store the entire contents of the file to make it easier to inspect the titles individually, as you'll do in the last part of the assignment. In the solution, I stored the lyrics data in a dictionary with two dimensions of keys: artist and song. The value was the file contents. A data frame would work equally well. 

For the Twitter data, we only need the description field for this assignment. Feel free all the descriptions read it into a data structure. In the solution, I stored the descriptions as a dictionary of lists, with the key being the artist. 




In [24]:
# Read in the lyrics data
lyric_df = []

for artist in os.listdir(os.path.join(data_location,lyrics_folder)):
    path = os.path.join(data_location,lyrics_folder, artist)

    for file in os.listdir(path):
        title = re.sub('.txt', '', file)
        title_b = re.sub('^cher|robyn_', '', title)
        title_c = re.sub('[0-9]$', '', title_b)

        song_path = path + '/' + file

        with open(song_path, 'r', encoding='utf-8') as s:
            lyrics = s.read()
            lyrics_c = re.sub(title_c, '', lyrics)

            lyric_df.append({
                'artist' :artist,
                'song_title': title_c,
                'lyrics': lyrics_c
            })
lyric_df = pd.DataFrame(lyric_df)


In [8]:
# Read in the twitter data
twitter_df= []

for artist in os.listdir(os.path.join(data_location,twitter_folder)):
    path = os.path.join(data_location,twitter_folder)

    for file in os.listdir(path):
        file_skip_sub = '_followers.txt'

        if file.endswith(file_skip_sub):
            continue

        twitter_path = os.path.join(data_location, twitter_folder, file)

        temp_df = pd.read_csv(twitter_path, sep='\t', encoding='utf-8', on_bad_lines='skip')
        desc = temp_df['description']
        
        for i in range(len(temp_df)):
            twitter_df.append({
                'artist': re.sub('_followers_data.txt', '', artist),
                'description': str(desc.iloc[i])
                })
twitter_df = pd.DataFrame(twitter_df)
twitter_df = twitter_df[twitter_df['description']!= 'nan']

        
    

## Data Cleaning

Now clean and tokenize your data. Remove punctuation chacters (available in the `punctuation` object in the `string` library), split on whitespace, fold to lowercase, and remove stopwords. Store your cleaned data, which must be accessible as an interable for `descriptive_stats`, in new objects or in new columns in your data frame. 



In [9]:
punctuation = set(punctuation) # speeds up comparison

In [54]:
def clean(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = text.lower().split()

    cleaned_tokens = [word for word in tokens if word not in sw]
    
    return cleaned_tokens

robyn_t = twitter_df[twitter_df['artist']!= 'cher']
cher_t = twitter_df[twitter_df['artist']!= 'robyn']

twitter_robyn = [clean(text) for text in robyn_t['description']]    
twitter_cher = [clean(text) for text in cher_t['description']]    

In [41]:
# create your clean lyrics data here
robyn_l = lyric_df[lyric_df['artist'] == 'robyn']
cher_l = lyric_df[lyric_df['artist'] == 'cher']

lyric_robyn = [clean(text) for text in robyn_l['lyrics']]    
lyric_cher = [clean(text) for text in cher_l['lyrics']]    

## Basic Descriptive Statistics

Call your `descriptive_stats` function on both your lyrics data and your twitter data and for both artists (four total calls). 

In [57]:
# calls to descriptive_stats here
descriptive_stats(str(lyric_robyn))

There are 134462 tokens in the data.
There are 50 unique tokens in the data.
There are 134462 characters in the data.
The lexical diversity is 0.000 in the data.


[134462, 50, 0.00037185227052996387, 134462]

In [45]:
lyric_df

,artist,song_title,lyrics
0,robyn,includemeout,"""Include Me Out""\n\n\n\nIt is really very simp..."
1,robyn,electric,"""Electric""\n\n\n\nElectric...\n\nIt's \nIt's a..."
2,robyn,beach2k2,"""Beach 2K20""\n\n\n\n(So you wanna go out?\nHow..."
3,robyn,lovekills,"""Love Kills""\n\n\n\nIf you're looking for love..."
4,robyn,timemachine,"""Time Machine""\n\n\n\nHey, what did I do?\nCan..."
...,...,...,...
415,cher,_takeitfromtheboys,"""Take It From The Boys""\n\n\n\nSo scared I nev..."
416,cher,_dreambaby,"""Dream Baby""\n\n\n\nI found the boy for me\nHe..."
417,cher,_pleasedonttellme,"""Please Don't Tell Me""\n\n\n\nYa shook the ove..."
418,cher,_ihopeyoufindit,"""I Hope You Find It""\n\n\n\nThese clouds aren'..."


Q: How do you think the "top 5 words" would be different if we left stopwords in the data? 

A: 

---

Q: What were your prior beliefs about the lexical diversity between the artists? Does the difference (or lack thereof) in lexical diversity between the artists conform to your prior beliefs? 

A: 



## Specialty Statistics

The descriptive statistics we have calculated are quite generic. You will now calculate a handful of statistics tailored to these data.

1. Ten most common emojis by artist in the twitter descriptions.
1. Ten most common hashtags by artist in the twitter descriptions.
1. Five most common words in song titles by artist. 
1. For each artist, a histogram of song lengths (in terms of number of tokens) 

We can use the `emoji` library to help us identify emojis and you have been given a function to help you.


In [6]:
assert(emoji.is_emoji("❤️"))
assert(not emoji.is_emoji(":-)"))

### Emojis 😁

What are the ten most common emojis by artist in the twitter descriptions? 


In [ ]:
# Your code here

### Hashtags

What are the ten most common hashtags by artist in the twitter descriptions? 


In [ ]:
# Your code here

### Song Titles

What are the five most common words in song titles by artist? The song titles should be on the first line of the lyrics pages, so if you have kept the raw file contents around, you will not need to re-read the data.


In [ ]:
# Your code here

### Song Lengths

For each artist, a histogram of song lengths (in terms of number of tokens). If you put the song lengths in a data frame with an artist column, matplotlib will make the plotting quite easy. An example is given to help you out. 


In [ ]:
num_replicates = 1000

df = pd.DataFrame({
    "artist" : ['Artist 1'] * num_replicates + ['Artist 2']*num_replicates,
    "length" : np.concatenate((np.random.poisson(125,num_replicates),np.random.poisson(150,num_replicates)))
})

df.groupby('artist')['length'].plot(kind="hist",density=True,alpha=0.5,legend=True)

Since the lyrics may be stored with carriage returns or tabs, it may be useful to have a function that can collapse whitespace, using regular expressions, and be used for splitting. 

Q: What does the regular expression `'\s+'` match on? 

A: 


In [ ]:
collapse_whitespace = re.compile(r'\s+')

def tokenize_lyrics(lyric) : 
    """strip and split on whitespace"""
    return([item.lower() for item in collapse_whitespace.split(lyric)])

In [ ]:
# Your lyric length comparison chart here. 